**Importing Libraries**

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

*Giving file paths for datasets*

In [ ]:
train_data = '/home/gautam/Desktop/Pneumonia_chest_xray_classification/datasets/train'
test_data = '/home/gautam/Desktop/Pneumonia_chest_xray_classification/datasets/test'
val_data = '/home/gautam/Desktop/Pneumonia_chest_xray_classification/datasets/val'

batch_size = 16

img_height = 500
img_width = 500

*Data Augmentation*

In [ ]:
image_gen = ImageDataGenerator(rotation_range=90, width_shift_range=0.1, height_shift_range=0.1, rescale=1/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_data_gen = ImageDataGenerator(rescale=1/255)

*Loading the Images*

In [ ]:
train = image_gen.flow_from_directory(train_data, target_size=(img_height, img_width), color_mode="grayscale", class_mode='binary', batch_size=batch_size)
test = test_data_gen.flow_from_directory(test_data, shuffle=False, target_size=(img_height, img_width), color_mode="grayscale", class_mode='binary', batch_size=batch_size)
valid = image_gen.flow_from_directory(val_data, target_size=(img_height, img_width), color_mode="grayscale", class_mode='binary', batch_size=batch_size)

*Visualising the Data*

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 10):
    plt.subplot(2, 5, i+1)
    for X_batch, Y_batch in train:
        image = X_batch[0]
        dic = {0:'NORMAL', 1:'PNEUMONIA'}
        plt.title(dic.get(Y_batch[0]))
        plt.axis('off')
        plt.imshow(np.squeeze(image), cmap='gray', interpolation='nearest')
        break
    plt.tight_layout()
    plt.show()

*CNN Architecture*

In [ ]:
cnn = Sequential()


cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))

cnn.add(Flatten())

cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'relu', units = 64))
cnn.add(Dense(activation = 'sigmoid', units = 1))

cnn.compile(optimizer = 'adam', loss = BinaryCrossentropy(), metrics = ['accuracy'])

cnn.summary()

*Interpreting Model Summary*

In [ ]:
Conv2D(filters=128, kernel_size=(3, 3),activation='relu', input_shape=(500, 500, 1))

MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)

*Visualize CNN model*

In [ ]:
plot_model(cnn, show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

*Fitting the model*

In [ ]:
early = EarlyStopping(monitor='val_loss', mode='auto', patience=5)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.003, min_lr=0.000001)

checkpoint_filepath = '/home/gautam/Desktop/Pneumonia_chest_xray_classification/checkpoint/checkpoint.h5'
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

callbacks_list = [early, learning_rate_reduction, checkpoint]

*Assigning Class Weights*

In [ ]:
weights = compute_class_weight(class_weight='balanced', classes=np.unique(train.classes), y=train.classes)
cw = dict(zip(np.unique(train.classes), weights))
print(cw)

*Fitting the model*

In [ ]:
cnn.fit(train, epochs=500, validation_data=valid, class_weight=cw, callbacks=callbacks_list)

*Evaluating the model*

In [ ]:
pd.DataFrame(cnn.history.history).plot()

*Loading Weights*

In [ ]:
cnn.load_weights(checkpoint_filepath)

*Testing Accuracy*

In [ ]:
test_loss, test_acc = cnn.evaluate(test)
print('Test Accuracy: ', test_acc*100, '%')

*Prediction*

In [ ]:
preds= cnn.predict(test, verbose=1)

predictions = preds.copy()
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0

*Creating a confusion matrix*

In [ ]:
cm = pd.DataFrame(data=confusion_matrix(test.classes, predictions, labels=[0, 1]), index=['Actual Normal', 'Actual Pneumonia'], columns=['Predicted Normal', 'Predicted Pneumonia'])

sns.heatmap(cm, annot=True, fmt='d')

*Printing the report*

In [ ]:
print(classification_report(y_true=test.classes, y_pred=predictions, target_names=['Normal', 'Pneumonia']))

*Concatenating the 'test' sequence and get the shape of 'x' and 'y'*

In [ ]:
test.reset()
x=np.concatenate([test.next()[0] for i in range(test.__len__())])
y=np.concatenate([test.next()[1] for i in range(test.__len__())])
print(x.shape)
print(y.shape)

*Saving the model*

In [ ]:
tf.keras.models.save_model(cnn, 'pneumonia.h5',save_format='h5')